# Import Libraries

In [88]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import PolynomialFeatures

# Import Datasets

In [9]:
games = pd.read_csv('input/games.csv')
details = pd.read_csv('input/games_details.csv')
teams = pd.read_csv('input/teams.csv')
players = pd.read_csv('input/players.csv')
rankings = pd.read_csv('input/ranking.csv')

/var/folders/20/pf71xw6d4q5ghhb3j4qdvltm0000gn/T/ipykernel_94299/2630087749.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  details = pd.read_csv('input/games_details.csv')


# Data Preparation

In [86]:
# Parse and store rankings
def get_labels(ranking):
    temp = ranking.copy(deep=False)
    temp = temp.groupby(['TEAM_ID','SEASON_ID'])[['G','W']].max()
    temp = pd.DataFrame(temp)
    temp.reset_index(inplace=True)
    drops = []
    for i in range(len(temp)):
        if temp.iloc[i,1] / 10000 > 2:
            temp.iloc[i,1] = temp.iloc[i,1] % 10000
        else:
            drops.append(i)
            continue
        if (temp.iloc[i,2] != 82):
            drops.append(i)
    for i in range(len(drops)):
        temp.drop([drops[i]], inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    temp.drop(columns=['G'], inplace=True)
    return temp

In [24]:
labels = get_labels(rankings)
labels

,TEAM_ID,SEASON_ID,W
0,1610612737,2002,35
1,1610612737,2003,28
2,1610612737,2004,13
3,1610612737,2005,26
4,1610612737,2006,30
...,...,...,...
501,1610612766,2015,48
502,1610612766,2016,36
503,1610612766,2017,36
504,1610612766,2018,39


## Get Features

In [27]:
def get_features(games, details):
    temp = pd.merge(games, details, how='left', left_on=['GAME_ID'], right_on = ['GAME_ID'])
    temp = temp[['TEAM_ID','SEASON','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']]
    temp = temp.groupby(['TEAM_ID','SEASON'])[['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']].sum()
    temp = pd.DataFrame(temp)
    next_season = []
    temp.reset_index(inplace=True)
    for i in range(len(temp)):
        next_season.append(temp.iloc[i,1] + 1)
    temp['NEXT_SEASON'] = next_season
    return temp

In [28]:
features = get_features(games, details)
features

,TEAM_ID,SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_SEASON
0,1.610613e+09,2003,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,2004
1,1.610613e+09,2004,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,2005
2,1.610613e+09,2005,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,2006
3,1.610613e+09,2006,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,2007
4,1.610613e+09,2007,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,1.610613e+09,2018,3495.0,7788.0,413.804,1043.0,2949.0,245.989,1619.0,2046.0,...,2980.0,3824.0,2029.0,625.0,435.0,1038.0,1679.0,9652.0,-330.0,2019
595,1.610613e+09,2019,2615.0,6015.0,297.664,839.0,2397.0,180.742,1135.0,1506.0,...,2249.0,3012.0,1668.0,466.0,297.0,1008.0,1333.0,7204.0,-2260.0,2020
596,1.610613e+09,2020,3210.0,7117.0,352.732,1095.0,2976.0,211.146,1299.0,1729.0,...,2704.0,3559.0,2173.0,640.0,386.0,1160.0,1485.0,8814.0,-899.0,2021
597,1.610613e+09,2021,3685.0,7945.0,398.470,1190.0,3301.0,218.024,1373.0,1859.0,...,2934.0,3877.0,2409.0,747.0,428.0,1129.0,1736.0,9933.0,-380.0,2022


## Processing of Remaining Datasets

In [29]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [30]:
def get_data(ranking, games, details):
    labels = get_labels(ranking)
    features = get_features(games, details)
    data = pd.merge(labels, features, how='left', left_on=['TEAM_ID','SEASON_ID'], right_on = ['TEAM_ID','NEXT_SEASON'])
    data.drop(columns=['SEASON_ID','SEASON'], inplace=True)
    data.dropna(inplace=True)
    data = swap_columns(data, 'W', 'NEXT_SEASON')
    data = data.astype({'NEXT_SEASON': 'int64'})
    data.rename(columns={'W' : 'NEXT_W'}, inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=['index'], inplace=True)
    return data

In [32]:
data = get_data(rankings, games, details)
data

,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2004,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,13
1,1610612737,2005,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,26
2,1610612737,2006,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,30
3,1610612737,2007,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,37
4,1610612737,2008,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,3191.0,7612.0,370.551,551.0,1736.0,154.944,1547.0,2057.0,...,3070.0,3965.0,1833.0,552.0,492.0,1017.0,1691.0,8480.0,-1430.0,48
443,1610612766,2016,3528.0,8122.0,425.323,987.0,2745.0,266.812,1859.0,2352.0,...,3403.0,4246.0,2027.0,691.0,485.0,1140.0,1758.0,9902.0,1195.0,36
444,1610612766,2017,3318.0,7589.0,387.285,866.0,2526.0,188.367,1741.0,2146.0,...,3111.0,3904.0,2024.0,621.0,425.0,1004.0,1486.0,9243.0,-175.0,36
445,1610612766,2018,3386.0,7537.0,390.534,872.0,2379.0,212.096,1738.0,2334.0,...,3078.0,3954.0,1869.0,594.0,393.0,1093.0,1502.0,9382.0,90.0,39


In [34]:
# Save output data
data.to_csv('output/nba_data.csv')

## Feature Scaling
We use z-score normalization.

In [35]:
def scale_data(data):
    temp = data.copy(deep=False)
    std_slc = StandardScaler()
    preprocess = std_slc.fit_transform(temp[['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']])
    data_scaled = pd.DataFrame(preprocess, columns=['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM'
                    ,'FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS'])
    data_scaled.insert(0,'TEAM_ID',temp[['TEAM_ID']])
    data_scaled.insert(1,'NEXT_SEASON',temp[['NEXT_SEASON']])
    data_scaled.insert(21,'NEXT_W',temp[['NEXT_W']])
    return data_scaled

In [36]:
data_scaled = scale_data(data)
data_scaled

,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2004,-1.411461,-1.264087,-1.498915,-1.106395,-1.073258,-1.705786,-0.471266,-0.614104,...,-0.976307,-0.816891,-1.158351,-0.629734,-0.391299,0.083757,-0.344346,-1.358318,-0.988451,13
1,1610612737,2005,-1.059369,-0.977735,-0.786667,-1.577773,-1.486932,-1.490412,-0.839727,-0.461249,...,-1.413032,-0.907023,-1.218349,-0.541993,-1.073390,0.063306,0.511068,-1.297323,-2.069663,26
2,1610612737,2006,-0.558896,-0.740009,-0.770914,-0.989593,-1.154715,-0.965047,0.455471,0.497565,...,-1.301051,-0.769454,-0.830124,-0.629734,-0.311722,0.663208,1.175506,-0.501689,-1.011865,30
3,1610612737,2007,-0.787756,-0.754867,-0.566770,-1.260740,-1.224992,-1.212437,0.682503,0.592057,...,-1.018299,-0.705412,-0.939533,-0.337266,0.404473,0.820000,1.016359,-0.668530,-1.011865,37
4,1610612737,2008,0.097503,0.166323,-0.304900,-1.047994,-1.082842,-1.304063,1.665067,1.481392,...,-0.018870,0.430724,0.316904,-0.015550,1.041091,0.894988,0.304177,0.245508,-0.603419,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,-0.571471,0.090682,-0.538987,-0.568274,-0.306604,-0.618743,-0.501040,-0.466808,...,0.499041,0.172188,-0.568954,-1.419399,0.540891,-1.790936,-0.988891,-0.650590,-0.741302,48
443,1610612766,2016,0.276064,0.779548,0.517056,1.250493,1.304966,1.772191,0.660172,0.353048,...,1.431282,0.838690,0.115733,-0.064295,0.461314,-0.952437,-0.722320,0.624936,0.624522,36
444,1610612766,2017,-0.252074,0.059616,-0.216343,0.745744,0.955180,0.095600,0.220995,-0.219461,...,0.613822,0.027502,0.105145,-0.746721,-0.220777,-1.879558,-1.804519,0.033817,-0.088308,36
445,1610612766,2018,-0.081058,-0.010621,-0.153700,0.770773,0.720392,0.602756,0.209830,0.303023,...,0.521438,0.146097,-0.441899,-1.009943,-0.584558,-1.272839,-1.740860,0.158499,0.049575,39


In [39]:
data_scaled.to_csv('output/nba_data_scaled.csv')

# Training

## Feature/Label Split

In [40]:
def split_data_X_y(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['TEAM_ID','NEXT_SEASON'], inplace=True)
    X = data.iloc[:,2:].copy(deep=False)
    X.drop(columns=['NEXT_W'], inplace=True)
    y = data.iloc[:,-1:].copy(deep=False)
    return X, y

## Train/Val Split

In [41]:
def split_data_train_test(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['TEAM_ID','NEXT_SEASON'], inplace=True)
    X = data.iloc[:,2:].copy(deep=False)
    X.drop(columns=['NEXT_W'], inplace=True)
    y = data.iloc[:,-1:].copy(deep=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)
    return X_train, X_test, y_train, y_test

In [42]:
def rmse(y_test, y_pred):
    return mean_squared_error(y_test, y_pred, squared=False)

In [43]:
X, y = split_data_X_y(data_scaled)

In [44]:
X_train, X_test, y_train, y_test = split_data_train_test(data_scaled)

## Cross Validation and K-Folds

In [45]:
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = make_scorer(rmse, greater_is_better=True)
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training RMSE": results['train_score'],
              "Mean RMSE": results['train_score'].mean(),
              "Validation RMSE": results['test_score'],
              "Mean Validation RMSE": results['test_score'].mean()
              }

In [95]:
def monte_carlo_validation(model, _X, _y, _cv=100):
      '''Function to perform Monte Carlo Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _cv: int, default=5
          Determines the number of folds for monte-carlo cross validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = make_scorer(rmse, greater_is_better=True)
      cv = ShuffleSplit(n_splits=_cv)
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training RMSE": results['train_score'],
              "Mean RMSE": results['train_score'].mean(),
              "Validation RMSE": results['test_score'],
              "Mean Validation RMSE": results['test_score'].mean()
              }

## Linear Regression

In [96]:
model_lr = LinearRegression()
print(monte_carlo_validation(model_lr, X, y))

{'Training RMSE': array([9.3930601 , 9.63471219, 9.63885972, 9.65184842, 9.69017381,
       9.61410484, 9.61440976, 9.55414044, 9.48840419, 9.65021455,
       9.58283194, 9.33087682, 9.57995024, 9.5976955 , 9.43452311,
       9.56822775, 9.36817794, 9.30680185, 9.60066324, 9.51354121,
       9.47909715, 9.73054473, 9.59037215, 9.35826965, 9.51836305,
       9.56390308, 9.44798896, 9.48973121, 9.51564659, 9.45308594,
       9.22727366, 9.49101779, 9.34002104, 9.6360465 , 9.4977611 ,
       9.50772325, 9.54237541, 9.51824292, 9.6220278 , 9.39474186,
       9.50827987, 9.44334567, 9.37732564, 9.63977816, 9.38288114,
       9.61178587, 9.74741315, 9.77590978, 9.43668752, 9.50085212,
       9.70870342, 9.47457464, 9.46184867, 9.13349739, 9.51975824,
       9.36839691, 9.64653892, 9.69309752, 9.44302914, 9.75518267,
       9.58976553, 9.53974701, 9.57787522, 9.5561682 , 9.70728046,
       9.73965569, 9.59621106, 9.57235298, 9.63587704, 9.66937077,
       9.50603918, 9.46077462, 9.58400108, 9

## Lasso Regression

In [97]:
model_lasso = linear_model.Lasso(alpha=0.1)
print(monte_carlo_validation(model_lasso, X, y))

{'Training RMSE': array([9.59009349, 9.65313088, 9.47556638, 9.58393258, 9.63920345,
       9.54250446, 9.58603821, 9.56336233, 9.5520094 , 9.57036057,
       9.65565544, 9.47777158, 9.75268324, 9.64070115, 9.54034052,
       9.63207499, 9.66362342, 9.66830982, 9.62992771, 9.79454304,
       9.71021537, 9.57979671, 9.58815429, 9.56705013, 9.64155293,
       9.38606442, 9.56883746, 9.68996145, 9.44800379, 9.51246948,
       9.42955956, 9.60879639, 9.6351519 , 9.63197129, 9.72984649,
       9.30652375, 9.71148574, 9.62093483, 9.59558132, 9.62159266,
       9.39899781, 9.65398296, 9.5274469 , 9.46374938, 9.69353577,
       9.69487145, 9.4759922 , 9.53529052, 9.59490833, 9.5980085 ,
       9.61414764, 9.31780756, 9.56553786, 9.65619013, 9.63888451,
       9.78099867, 9.52303114, 9.63843425, 9.65694511, 9.56657607,
       9.24352714, 9.61429049, 9.25787674, 9.5572747 , 9.66927305,
       9.3508142 , 9.53252825, 9.19019251, 9.69175263, 9.29668512,
       9.41315916, 9.50505427, 9.61958433, 9

## Support Vector Machine Regression

In [99]:
model_svm = SVR(kernel = 'linear').fit(X_train, y_train)
print(monte_carlo_validation(model_svm, X, y))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/uti

{'Training RMSE': array([9.54158229, 9.70309392, 9.49067158, 9.62312583, 9.52976564,
       9.31623932, 9.74313147, 9.54154262, 9.62404457, 9.69380309,
       9.72263737, 9.593229  , 9.55499563, 9.57006321, 9.51301499,
       9.69912613, 9.57759533, 9.54162581, 9.65534002, 9.58417079,
       9.74048089, 9.58194476, 9.52641401, 9.63732066, 9.60309668,
       9.52287322, 9.53114968, 9.60524864, 9.71405926, 9.62775402,
       9.6358613 , 9.72111898, 9.77824873, 9.76899883, 9.60894743,
       9.62962108, 9.31560318, 9.8007639 , 9.60708474, 9.65163222,
       9.55606937, 9.77565493, 9.57847467, 9.7134532 , 9.65102552,
       9.51670816, 9.46884   , 9.81358159, 9.76086784, 9.52661442,
       9.54923556, 9.64134315, 9.69169348, 9.6458097 , 9.71333721,
       9.5692125 , 9.48373457, 9.08305688, 9.77247555, 9.72017582,
       9.53574106, 9.60754503, 9.78809831, 9.83683994, 9.75340892,
       9.59219553, 9.40253126, 9.54836541, 9.7952693 , 9.46365026,
       9.71146391, 9.69447645, 9.61950687, 9

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/uti

## Random Forest Regression

In [49]:
model_tree = RandomForestRegressor(random_state=2023)
print(cross_validation(model_tree, X, y))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks

{'Training RMSE': array([3.68165906, 3.98833593, 3.87518098, 3.88696644, 3.97655774]), 'Mean RMSE': 3.8817400316406085, 'Validation RMSE': array([11.80599288, 10.20663352, 10.62018912, 10.45245619,  8.94105146]), 'Mean Validation RMSE': 10.40526463139284}


# Feature Selection

## Lasso Regression Coefficient

In [100]:
model_lasso = linear_model.Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)
y_pred = model_lasso.predict(X_test)
print(mean_squared_error(y_test, y_pred, squared=False))
print(model_lasso.coef_)

10.108649119789948
[ 0.          0.         -0.5065314   0.          0.         -0.66973622
  0.         -0.         -0.19109535  0.          0.72932052  0.17580654
 -0.          0.94379254  0.          0.4820294   0.01080316  0.
  7.50363662]


In [51]:
def feature_selection(data):
    temp = data.copy(deep=False)
    temp.drop(columns=['FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','AST','BLK','PTS'], inplace=True)
    return temp;

In [101]:
data_selected = feature_selection(data_scaled)
X_new, y_new = split_data_X_y(data_selected)
X_new_train, X_new_test, y_new_train, y_new_test = split_data_train_test(data_selected)
data_selected

,TEAM_ID,NEXT_SEASON,FG_PCT,FG3_PCT,FT_PCT,DREB,REB,STL,TO,PF,PLUS_MINUS,NEXT_W
0,1610612737,2004,-1.498915,-1.705786,-0.861508,-0.976307,-0.816891,-0.629734,0.083757,-0.344346,-0.988451,13
1,1610612737,2005,-0.786667,-1.490412,-0.701850,-1.413032,-0.907023,-0.541993,0.063306,0.511068,-2.069663,26
2,1610612737,2006,-0.770914,-0.965047,0.197828,-1.301051,-0.769454,-0.629734,0.663208,1.175506,-1.011865,30
3,1610612737,2007,-0.566770,-1.212437,0.186456,-1.018299,-0.705412,-0.337266,0.820000,1.016359,-1.011865,37
4,1610612737,2008,-0.304900,-1.304063,1.040891,-0.018870,0.430724,-0.015550,0.894988,0.304177,-0.603419,47
...,...,...,...,...,...,...,...,...,...,...,...,...
442,1610612766,2015,-0.538987,-0.618743,0.104340,0.499041,0.172188,-1.419399,-1.790936,-0.988891,-0.741302,48
443,1610612766,2016,0.517056,1.772191,1.550633,1.431282,0.838690,-0.064295,-0.952437,-0.722320,0.624522,36
444,1610612766,2017,-0.216343,0.095600,1.398407,0.613822,0.027502,-0.746721,-1.879558,-1.804519,-0.088308,36
445,1610612766,2018,-0.153700,0.602756,0.632825,0.521438,0.146097,-1.009943,-1.272839,-1.740860,0.049575,39


# Retraining

## Linear Regression

In [102]:
model_lr_new = LinearRegression()
print(cross_validation(model_lr_new, X_new, y_new))

{'Training RMSE': array([9.22851245, 9.71357778, 9.53587644, 9.53971929, 9.84449558]), 'Mean RMSE': 9.572436305916387, 'Validation RMSE': array([11.28666531,  9.32700823, 10.06012399, 10.09426453,  8.68503644]), 'Mean Validation RMSE': 9.890619698782483}


## Lasso Regression

In [103]:
model_lasso_new = linear_model.Lasso(alpha=0.1)
print(cross_validation(model_lasso_new, X_new, y_new))

{'Training RMSE': array([9.2407124 , 9.72543766, 9.5492515 , 9.55080027, 9.85387941]), 'Mean RMSE': 9.584016249349592, 'Validation RMSE': array([11.10286677,  9.3476289 , 10.00699856,  9.99603348,  8.7137333 ]), 'Mean Validation RMSE': 9.833452201944691}


## Support Vector Machine Regression

In [104]:
model_svm_new = SVR(kernel = 'linear')
print(cross_validation(model_svm_new, X_new, y_new))

{'Training RMSE': array([9.25062681, 9.7462161 , 9.59932656, 9.57569686, 9.89479155]), 'Mean RMSE': 9.613331575118114, 'Validation RMSE': array([11.26103375,  9.35133074, 10.27951509, 10.12513306,  8.65664761]), 'Mean Validation RMSE': 9.934732049648769}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/uti

## Random Forest Regression

In [105]:
model_tree_new = RandomForestRegressor(random_state=2023)
print(cross_validation(model_tree_new, X_new, y_new))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks

{'Training RMSE': array([3.6610285 , 3.96493372, 3.89475274, 3.89762379, 3.94863501]), 'Mean RMSE': 3.873394753061979, 'Validation RMSE': array([11.83831257,  9.89979781, 10.75042539, 10.37173899,  8.95092118]), 'Mean Validation RMSE': 10.362239186319268}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


# Testing Demo

In our demo, we just use the Lasso Regression Model to predict the number of game wins in 2018, using team statistics from 2017. This is because data in later years are somewhat incomplete.

In [106]:
model_lasso_new = linear_model.Lasso(alpha=0.1)
model_lasso_new.fit(X_new_train, y_new_train)

Lasso(alpha=0.1)

In [107]:
def get_2020(data):
    temp = data.copy(deep=False)
    drop_list = []
    for i in range(len(temp)):
        if temp.iloc[i, 1] != 2021:
            drop_list.append(i)
            
    temp.drop(drop_list, inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    return temp

In [108]:
data_2020 = get_2020(data_scaled)
data_2020

,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
0,1610612737,2021,1.485750,1.581874,0.927485,2.322564,2.360713,1.814359,1.036077,0.519798,...,1.677640,1.493332,1.238056,-0.093542,0.052060,-0.052584,0.232561,1.808969,0.482476,43
1,1610612738,2021,0.271034,0.150114,0.140157,1.851187,1.911901,1.184139,-1.059315,-1.189392,...,-0.063663,-0.247637,-0.067791,-0.249525,0.324896,-0.713840,-0.758128,0.336104,-0.047723,51
2,1610612739,2021,-1.104638,-1.192499,-1.273427,0.407854,0.635741,0.253545,-1.386837,-1.275547,...,-0.990304,-1.288898,-0.508956,-0.814964,-0.982445,-0.829730,-1.983559,-1.034503,-1.673184,44
3,1610612740,2021,-0.272193,-0.811597,-0.787843,0.562198,0.733170,-0.286097,-0.754125,-0.491820,...,-0.200839,-0.361488,-0.036027,-1.000194,-1.221176,-1.402364,-2.154642,-0.255015,0.079754,36
4,1610612741,2021,-0.174110,-0.602236,-0.505997,1.296380,1.308161,0.800091,-1.989774,-2.120415,...,-0.086059,-0.629512,0.433372,-1.565633,-1.130231,-0.625218,-1.705052,-0.324981,-0.218907,46
5,1610612742,2021,-0.083573,-0.392874,0.062012,1.955474,1.958219,0.946858,-1.208189,-1.308897,...,-0.206438,-0.729131,-0.371313,-1.789859,-1.164335,-2.315851,-1.279334,0.069697,0.415876,52
6,1610612743,2021,1.191502,0.679595,0.712331,2.043075,1.897526,2.402967,-0.906720,-1.111576,...,0.445850,0.267063,1.506285,0.179429,-0.436772,-0.829730,-0.718342,1.070742,0.961165,48
7,1610612744,2021,-0.211835,-0.422590,-0.324200,2.043075,2.006135,1.246206,-1.159805,-1.333909,...,-0.158846,-0.954461,0.676895,-0.308019,-0.470877,-0.570681,-0.817808,0.008701,0.000146,53
8,1610612745,2021,-0.815420,-0.775127,-1.331115,1.692670,2.042871,1.049020,-1.416612,-1.300559,...,-0.693555,-1.231973,-0.519544,-0.863709,-0.516349,-1.027425,-1.482247,-0.559096,-1.404702,20
9,1610612746,2021,1.234256,0.857890,1.348884,2.760570,2.274464,2.889925,-0.259121,-0.797529,...,1.078542,0.632335,0.934535,-0.308019,-0.482245,-0.945620,-0.304560,1.411600,1.220281,42


In [109]:
data_2020_selected = feature_selection(data_2020)
X_2020, y_2021 = split_data_X_y(data_2020_selected)

In [110]:
y_2021_pred = model_lasso_new.predict(X_2020)
print(np.round(y_2021_pred))

[44. 40. 27. 41. 38. 41. 47. 40. 29. 48. 44. 37. 54. 32. 48. 43. 25. 39.
 51. 50. 40. 34. 35. 24. 39. 53. 42. 35. 33. 37.]


In [111]:
team_id_name = {}
for i in range(len(teams)):
    team_id_name[teams.iloc[i, 1]] = teams.iloc[i, 5]
print(team_id_name)

{1610612737: 'Hawks', 1610612738: 'Celtics', 1610612740: 'Pelicans', 1610612741: 'Bulls', 1610612742: 'Mavericks', 1610612743: 'Nuggets', 1610612745: 'Rockets', 1610612746: 'Clippers', 1610612747: 'Lakers', 1610612748: 'Heat', 1610612749: 'Bucks', 1610612750: 'Timberwolves', 1610612751: 'Nets', 1610612752: 'Knicks', 1610612753: 'Magic', 1610612754: 'Pacers', 1610612755: '76ers', 1610612756: 'Suns', 1610612757: 'Trail Blazers', 1610612758: 'Kings', 1610612759: 'Spurs', 1610612760: 'Thunder', 1610612761: 'Raptors', 1610612762: 'Jazz', 1610612763: 'Grizzlies', 1610612764: 'Wizards', 1610612765: 'Pistons', 1610612766: 'Hornets', 1610612739: 'Cavaliers', 1610612744: 'Warriors'}


In [112]:
name_list = []
for i in range(len(data_2020)):
    name_list.append(team_id_name[data_2020.iloc[i, 0]])
print(name_list)

['Hawks', 'Celtics', 'Cavaliers', 'Pelicans', 'Bulls', 'Mavericks', 'Nuggets', 'Warriors', 'Rockets', 'Clippers', 'Lakers', 'Heat', 'Bucks', 'Timberwolves', 'Nets', 'Knicks', 'Magic', 'Pacers', '76ers', 'Suns', 'Trail Blazers', 'Kings', 'Spurs', 'Thunder', 'Raptors', 'Jazz', 'Grizzlies', 'Wizards', 'Pistons', 'Hornets']


In [113]:
prediction_dict_2021 = {"team_name": name_list, "wins_pred_2021": np.round(y_2021_pred), 'wins_2021': y_2021['NEXT_W']}
prediction_df_2021 = pd.DataFrame(prediction_dict_2021)
prediction_df_2021.sort_values(by='wins_pred_2021', ascending=False, inplace=True)
prediction_df_2021 = prediction_df_2021.astype({'wins_pred_2021': 'int64'})
prediction_df_2021.reset_index(inplace=True)
prediction_df_2021.drop(columns=['index'], inplace=True)

In [114]:
prediction_df_2021

,team_name,wins_pred_2021,wins_2021
0,Bucks,54,51
1,Jazz,53,49
2,76ers,51,51
3,Suns,50,64
4,Clippers,48,42
5,Nets,48,44
6,Nuggets,47,48
7,Hawks,44,43
8,Lakers,44,33
9,Knicks,43,37


In [85]:
prediction_df_2021.to_csv('output/nba_data_2021_prediction.csv')

# Sports Betting Live Spread Prediction

We first pull, process, and store all historical betting odds.

In [24]:
import random
import time
import pandas as pd
import sqlite3
import os
import sys
import re
import requests

from datetime import datetime, timedelta
from tqdm import tqdm
from sbrscrape import Scoreboard
import random
import time
import pandas as pd
import sqlite3
import os
import sys
import re
import requests

from datetime import datetime, timedelta
from tqdm import tqdm
from sbrscrape import Scoreboard
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from .Dictionaries import team_index_07, team_index_08, team_index_12, team_index_13, team_index_14, team_index_current

In [4]:
year = [2022, 2023]
season = ["2022-23"]

month = [10, 11, 12, 1, 2, 3, 4, 5, 6]
days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

begin_year_pointer = year[0]
end_year_pointer = year[0]
count = 0

sportsbook='fanduel'
df_data = []

In [6]:
con = sqlite3.connect("Data/odds.sqlite")

for season1 in tqdm(season):
    teams_last_played = {}
    for month1 in tqdm(month):
        if month1 == 1:
            count += 1
            end_year_pointer = year[count]
        for day1 in tqdm(days):
            if month1 == 10 and day1 < 19:
                continue
            if month1 in [4,6,9,11] and day1 > 30:
                continue
            if month1 == 2 and day1 > 28:
                continue
            if end_year_pointer == datetime.now().year:
                if month1 == datetime.now().month and day1 >= datetime.now().day:
                    continue
                if month1 > datetime.now().month:
                    continue
            sb = Scoreboard(date=f"{end_year_pointer}-{month1:02}-{day1:02}")
            if not hasattr(sb, "games"):
                continue
            for game in sb.games:
                if game['home_team'] not in teams_last_played:
                    teams_last_played[game['home_team']] = get_date(f"{season1}-{month1:02}{day1:02}")
                    home_games_rested = timedelta(days=7)
                else:
                    current_date = get_date(f"{season1}-{month1:02}{day1:02}")
                    home_games_rested = current_date - teams_last_played[game['home_team']]
                    teams_last_played[game['home_team']] = current_date

                if game['away_team'] not in teams_last_played:
                    teams_last_played[game['away_team']] = get_date(f"{season1}-{month1:02}{day1:02}")
                    away_games_rested = timedelta(days=7)
                else:
                    current_date = get_date(f"{season1}-{month1:02}{day1:02}")
                    away_games_rested = current_date - teams_last_played[game['away_team']]
                    teams_last_played[game['away_team']] = current_date
                
                try:
                    df_data.append({
                        'Unnamed: 0': 0,
                        'Date': f"{season1}-{month1:02}{day1:02}",
                        'Home': game['home_team'],
                        'Away': game['away_team'],
                        'OU': game['total'][sportsbook],
                        'Spread': game['away_spread'][sportsbook],
                        'ML_Home': game['home_ml'][sportsbook],
                        'ML_Away': game['away_ml'][sportsbook],
                        'Points': game['away_score'] + game['home_score'],
                        'Win_Margin': game['home_score'] - game['away_score'],
                        'Days_Rest_Home': home_games_rested.days,
                        'Days_Rest_Away': away_games_rested.days
                    })
                except KeyError:
                    print(f"No {sportsbook} odds data found for game: {game}")
            time.sleep(random.randint(1, 3))
    begin_year_pointer = year[count]

    df = pd.DataFrame(df_data,)
    df.to_sql(f"odds_{season1}", con, if_exists="replace")
con.close()

  0%|          | 0/1 [00:00<?, ?it/s]













100%|██████████| 31/31 [00:34<00:00,  1.10s/it]































100%|██████████| 31/31 [01:18<00:00,  2.54s/it]
































100%|██████████| 31/31 [01:19<00:00,  2.56s/it]
































100%|██████████| 31/31 [01:18<00:00,  2.54s/it]





























100%|██████████| 31/31 [01:17<00:00,  2.51s/it]
































100%|██████████| 31/31 [01:29<00:00,  2.88s/it]































100%|██████████| 31/31 [01:18<00:00,  2.54s/it]
































100%|██████████| 31/31 [01:22<00:00,  2.65s/it]


















100%|██████████| 1/1 [10:38<00:00, 638.59s/it]


We now go about leveraging these betting spreads to make a more informed bet using the predicted output from the ML model above.

In [7]:
def payout(odds):
    if odds > 0:
        return odds
    else:
        return (100 / (-1 * odds)) * 100

In [8]:
def expected_value(Pwin, odds):
    Ploss = 1 - Pwin
    Mwin = payout(odds)
    return round((Pwin * Mwin) - (Ploss * 100), 2)

In [9]:
def american_to_decimal(american_odds):
    """
    Converts American odds to decimal odds (European odds).
    """
    if american_odds >= 100:
        decimal_odds = (american_odds / 100)
    else:
        decimal_odds = (100 / abs(american_odds))
    return round(decimal_odds, 2)

def calculate_kelly_criterion(american_odds, model_prob):
    """
    Calculates the fraction of the bankroll to be wagered on each bet
    """
    decimal_odds = american_to_decimal(american_odds)
    bankroll_fraction = round((100 * (decimal_odds * model_prob - (1 - model_prob))) / decimal_odds, 2)
    return bankroll_fraction if bankroll_fraction > 0 else 0

In [10]:
games_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/57.0.2987.133 Safari/537.36',
    'Dnt': '1',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en',
    'origin': 'http://stats.nba.com',
    'Referer': 'https://github.com'
}

data_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nba.com/',
    'Connection': 'keep-alive'
}

In [11]:
def get_json_data(url):
    raw_data = requests.get(url, headers=data_headers)
    try:
        json = raw_data.json()
    except Exception as e:
        print(e)
        return {}
    return json.get('resultSets')


def get_todays_games_json(url):
    raw_data = requests.get(url, headers=games_header)
    json = raw_data.json()
    return json.get('gs').get('g')


def to_data_frame(data):
    try:
        data_list = data[0]
    except Exception as e:
        print(e)
        return pd.DataFrame(data={})
    return pd.DataFrame(data=data_list.get('rowSet'), columns=data_list.get('headers'))


def create_todays_games(input_list):
    games = []
    for game in input_list:
        home = game.get('h')
        away = game.get('v')
        home_team = home.get('tc') + ' ' + home.get('tn')
        away_team = away.get('tc') + ' ' + away.get('tn')
        games.append([home_team, away_team])
    return games


def create_todays_games_from_odds(input_dict):
    games = []
    for game in input_dict.keys():
        home_team, away_team = game.split(":")
        if home_team not in team_index_current or away_team not in team_index_current:
            continue
        games.append([home_team, away_team])
    return games

def get_date(date_string):
    year1,month,day = re.search(r'(\d+)-\d+-(\d\d)(\d\d)', date_string).groups()
    year = year1 if int(month) > 8 else int(year1) + 1
    return datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')

In [16]:
class SbrOddsProvider:
    
    """ Abbreviations dictionary for team location which are sometimes saved with abbrev instead of full name. 
    Moneyline options name require always full name
    Returns:
        string: Full location name
    """    

    def __init__(self, sportsbook="fanduel"):
        sb = Scoreboard(sport="NBA")
        self.games = sb.games if hasattr(sb, 'games') else []
        self.sportsbook = sportsbook

    
    def get_odds(self):
        """Function returning odds from Sbr server's json content

        Returns:
            dictionary: [home_team_name + ':' + away_team_name: { home_team: money_line_odds, away_team: money_line_odds }, under_over_odds: val]
        """
        dict_res = {}
        for game in self.games:
            # Get team names
            home_team_name = game['home_team'].replace("Los Angeles Clippers", "LA Clippers")
            away_team_name = game['away_team'].replace("Los Angeles Clippers", "LA Clippers")
            
            money_line_home_value = money_line_away_value = totals_value = None

            # Get money line bet values
            if self.sportsbook in game['home_ml']:
                money_line_home_value = game['home_ml'][self.sportsbook]
            if self.sportsbook in game['away_ml']:
                money_line_away_value = game['away_ml'][self.sportsbook]
            
            # Get totals bet value
            if self.sportsbook in game['total']:
                totals_value = game['total'][self.sportsbook]
            
            dict_res[home_team_name + ':' + away_team_name] =  { 
                'under_over_odds': totals_value,
                home_team_name: { 'money_line_odds': money_line_home_value }, 
                away_team_name: { 'money_line_odds': money_line_away_value }
            }
        return dict_res

In [17]:
from .Dictionaries import team_index_07, team_index_08, team_index_12, team_index_13, team_index_14, team_index_current

season_array = ["2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]

df = pd.DataFrame
scores = []
win_margin = []
OU = []
OU_Cover = []
games = []
days_rest_away = []
days_rest_home = []
teams_con = sqlite3.connect("Data/teams.sqlite")
odds_con = sqlite3.connect("Data/odds.sqlite")

for season in tqdm(season_array):
    odds_df = pd.read_sql_query(f"select * from \"odds_{season}\"", odds_con, index_col="index")
    team_table_str = "teams_{}-{}-" + season
    year_count = 0

    for row in odds_df.itertuples():
        home_team = row[3]
        away_team = row[4]

        date = row[2]
        date_array = date.split('-')
        if not date_array or len(date_array) < 2:
            continue
        year = date_array[0] + '-' + date_array[1]
        month = date_array[2][:2]
        day = date_array[2][2:]

        if month[0] == '0':
            month = month[1:]
        if day[0] == '0':
            day = day[1:]
        if int(month) == 1:
            year_count = 1
        end_year_pointer = int(date_array[0]) + year_count
        if end_year_pointer == datetime.now().year:
            if int(month) == datetime.now().month and int(day) >= datetime.now().day:
                continue
            if int(month) > datetime.now().month:
                continue

        team_df = pd.read_sql_query(f"select * from \"teams_{year}-{month}-{day}\"", teams_con, index_col="index")
        if len(team_df.index) == 30:
            scores.append(row[9])
            OU.append(row[5])
            days_rest_home.append(row[11])
            days_rest_away.append(row[12])
            if row[10] > 0:
                win_margin.append(1)
            else:
                win_margin.append(0)

            if row[9] < row[5]:
                OU_Cover.append(0)
            elif row[9] > row[5]:
                OU_Cover.append(1)
            elif row[9] == row[5]:
                OU_Cover.append(2)

            if season == '2007-08':
                home_team_series = team_df.iloc[team_index_07.get(home_team)]
                away_team_series = team_df.iloc[team_index_07.get(away_team)]
            elif season == '2008-09' or season == "2009-10" or season == "2010-11" or season == "2011-12":
                home_team_series = team_df.iloc[team_index_08.get(home_team)]
                away_team_series = team_df.iloc[team_index_08.get(away_team)]
            elif season == "2012-13":
                home_team_series = team_df.iloc[team_index_12.get(home_team)]
                away_team_series = team_df.iloc[team_index_12.get(away_team)]
            elif season == '2013-14':
                home_team_series = team_df.iloc[team_index_13.get(home_team)]
                away_team_series = team_df.iloc[team_index_13.get(away_team)]
            elif season == '2022-23':
                home_team_series = team_df.iloc[team_index_current.get(home_team)]
                away_team_series = team_df.iloc[team_index_current.get(away_team)]
            else:
                try:
                    home_team_series = team_df.iloc[team_index_14.get(home_team)]
                    away_team_series = team_df.iloc[team_index_14.get(away_team)]
                except Exception as e:
                    print(home_team)
                    raise e
            game = pd.concat([home_team_series, away_team_series.rename(
                index={col:f"{col}.1" for col in team_df.columns.values}
            )])
            games.append(game)
odds_con.close()
teams_con.close()
season = pd.concat(games, ignore_index=True, axis=1)
season = season.T
frame = season.drop(columns=['TEAM_ID', 'CFID', 'CFPARAMS', 'Unnamed: 0', 'Unnamed: 0.1', 'CFPARAMS.1', 'TEAM_ID.1', 'CFID.1'])
frame['Score'] = np.asarray(scores)
frame['Home-Team-Win'] = np.asarray(win_margin)
frame['OU'] = np.asarray(OU)
frame['OU-Cover'] = np.asarray(OU_Cover)
frame['Days-Rest-Home'] = np.asarray(days_rest_home)
frame['Days-Rest-Away'] = np.asarray(days_rest_away)
# fix types
for field in frame.columns.values:
    if 'TEAM_' in field  or 'Date' in field or field not in frame:
        continue
    frame[field] = frame[field].astype(float)
con = sqlite3.connect("Data/dataset.sqlite")
frame.to_sql("dataset_2012-23", con, if_exists="replace")
con.close()

ImportError: attempted relative import with no known parent package

Pull today's betting odds to make a betting decision.

In [ ]:
todays_games_url = 'https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2022/scores/00_todays_scores.json'
data_url = 'https://stats.nba.com/stats/leaguedashteamstats?' \
           'Conference=&DateFrom=&DateTo=&Division=&GameScope=&' \
           'GameSegment=&LastNGames=0&LeagueID=00&Location=&' \
           'MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&' \
           'PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&' \
           'PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&' \
           'Season=2022-23&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
           'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

In [ ]:
def createTodaysGames(games, df, odds):
    match_data = []
    todays_games_uo = []
    home_team_odds = []
    away_team_odds = []
    # todo: get the days rest for current games
    home_team_days_rest = []
    away_team_days_rest = []

    for game in games:
        home_team = game[0]
        away_team = game[1]
        if home_team not in team_index_current or away_team not in team_index_current:
            continue
        if odds is not None:
            game_odds = odds[home_team + ':' + away_team]
            todays_games_uo.append(game_odds['under_over_odds'])
            
            home_team_odds.append(game_odds[home_team]['money_line_odds'])
            away_team_odds.append(game_odds[away_team]['money_line_odds'])

        else:
            todays_games_uo.append(input(home_team + ' vs ' + away_team + ': '))

            home_team_odds.append(input(home_team + ' odds: '))
            away_team_odds.append(input(away_team + ' odds: '))
        
        # calculate days rest for both teams
        dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M')
        schedule_df = pd.read_csv('Data/nba-2022-UTC.csv', parse_dates=['Date'], date_parser=dateparse)
        home_games = schedule_df[(schedule_df['Home Team'] == home_team) | (schedule_df['Away Team'] == home_team)]
        away_games = schedule_df[(schedule_df['Home Team'] == away_team) | (schedule_df['Away Team'] == away_team)]
        last_home_date = home_games.loc[schedule_df['Date'] <= datetime.today()].sort_values('Date',ascending=False).head(1)['Date'].iloc[0]
        last_away_date = away_games.loc[schedule_df['Date'] <= datetime.today()].sort_values('Date',ascending=False).head(1)['Date'].iloc[0]
        home_days_off = timedelta(days=1) + datetime.today() - last_home_date
        away_days_off = timedelta(days=1) + datetime.today() - last_away_date
        # print(f"{away_team} days off: {away_days_off.days} @ {home_team} days off: {home_days_off.days}")

        home_team_days_rest.append(home_days_off.days)
        away_team_days_rest.append(away_days_off.days)
        home_team_series = df.iloc[team_index_current.get(home_team)]
        away_team_series = df.iloc[team_index_current.get(away_team)]
        stats = pd.concat([home_team_series, away_team_series])
        stats['Days-Rest-Home'] = home_days_off.days
        stats['Days-Rest-Away'] = away_days_off.days
        match_data.append(stats)

    games_data_frame = pd.concat(match_data, ignore_index=True, axis=1)
    games_data_frame = games_data_frame.T

    frame_ml = games_data_frame.drop(columns=['TEAM_ID', 'TEAM_NAME'])
    data = frame_ml.values
    data = data.astype(float)

    return data, todays_games_uo, frame_ml, home_team_odds, away_team_odds

In [ ]:
def main():
    odds = None
    if args.odds:
        odds = SbrOddsProvider(sportsbook=args.odds).get_odds()
        games = create_todays_games_from_odds(odds)
        if len(games) == 0:
            print("No games found.")
            return
        if((games[0][0]+':'+games[0][1]) not in list(odds.keys())):
            print(games[0][0]+':'+games[0][1])
            print(Fore.RED, "--------------Games list not up to date for todays games!!! Scraping disabled until list is updated.--------------")
            print(Style.RESET_ALL)
            odds = None
        else:
            print(f"------------------{args.odds} odds data------------------")
            for g in odds.keys():
                home_team, away_team = g.split(":")
                print(f"{away_team} ({odds[g][away_team]['money_line_odds']}) @ {home_team} ({odds[g][home_team]['money_line_odds']})")
    else:
        data = get_todays_games_json(todays_games_url)
        games = create_todays_games(data)
    data = get_json_data(data_url)
    df = to_data_frame(data)
    data, todays_games_uo, frame_ml, home_team_odds, away_team_odds = createTodaysGames(games, df, odds)
    if args.nn:
        print("------------Neural Network Model Predictions-----------")
        data = tf.keras.utils.normalize(data, axis=1)
        NN_Runner.nn_runner(data, todays_games_uo, frame_ml, games, home_team_odds, away_team_odds, args.kc)
        print("-------------------------------------------------------")
    if args.xgb:
        print("---------------XGBoost Model Predictions---------------")
        XGBoost_Runner.xgb_runner(data, todays_games_uo, frame_ml, games, home_team_odds, away_team_odds, args.kc)
        print("-------------------------------------------------------")
    if args.A:
        print("---------------XGBoost Model Predictions---------------")
        XGBoost_Runner.xgb_runner(data, todays_games_uo, frame_ml, games, home_team_odds, away_team_odds, args.kc)
        print("-------------------------------------------------------")
        data = tf.keras.utils.normalize(data, axis=1)
        print("------------Neural Network Model Predictions-----------")
        NN_Runner.nn_runner(data, todays_games_uo, frame_ml, games, home_team_odds, away_team_odds, args.kc)
        print("-------------------------------------------------------")

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Model to Run')
    parser.add_argument('-xgb', action='store_true', help='Run with XGBoost Model')
    parser.add_argument('-nn', action='store_true', help='Run with Neural Network Model')
    parser.add_argument('-A', action='store_true', help='Run all Models')
    parser.add_argument('-odds', help='Sportsbook to fetch from. (fanduel, draftkings, betmgm, pointsbet, caesars, wynn, bet_rivers_ny')
    parser.add_argument('-kc', action='store_true', help='Calculates percentage of bankroll to bet based on model edge')
    args = parser.parse_args()
    main()